# 프로젝트: 얼굴인식 후 캐릭터 씌우기

### Face Detection(얼굴 객체를 인식) vs Face Recognition(누구의 얼굴인지를 확인)

In [ ]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    alpha = overlay_image[:,:,3]
    mask_image = alpha / 255

    for c in range(0,3):
        image[y-h:y+h,x-w:x+w,c] = (overlay_image[:,:,c] * mask_image) + (image[y-h:y+h,x-w:x+w,c] * (1 - mask_image))


#얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection #얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

#동영상 파일 열기(웹캠은 걍 0(기본))
cap = cv2.VideoCapture('face_video.mp4')

img_right_eye = cv2.imread('right_eye.png',cv2.IMREAD_UNCHANGED)
img_left_eye = cv2.imread('left_eye.png',cv2.IMREAD_UNCHANGED)

img_right_eye = cv2.resize(img_right_eye,(100,100))
img_left_eye = cv2.resize(img_left_eye,(100,100))


with mp_face_detection.FaceDetection(
    model_selection = 0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image, = cap.read()
        if not success:
            break #if in webcam, use continue
            
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)
    
        # Draw the face detection annotation on the image
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            #6개 특징: 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀(귀 구슬점)
            for detection in results.detections:
                #mp_drawing.draw_detection(image, detection)
                #특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                #nose_tip = keypoints[2]

                h,w,_ = image.shape#height, width, channel: 이미지로부터 세로, 가로 크기 가져옴(_: 안쓴다는 의미)
                right_eye = (int(right_eye.x * w)-20, int(right_eye.y * h)-100)
                left_eye = (int(left_eye.x * w)+20, int(left_eye.y*h)-100)
                #nose_tip = (int(nose_tip.x*w), int(nose_tip.y*h))
                
                # cv2.circle(image, right_eye, 50,(255,0,0),10,cv2.LINE_AA) #파란색
                # cv2.circle(image, left_eye, 50,(0,255,0),10,cv2.LINE_AA)
                # cv2.circle(image, nose_tip, 50,(0,255,255),10,cv2.LINE_AA)
                
                #
                # image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = img_right_eye
                # image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = img_left_eye
                overlay(image, *right_eye, 50,50, img_right_eye)
                overlay(image, *left_eye, 50,50, img_left_eye)




        
        cv2.imshow('Face Detection', cv2.resize(image,None, fx=0.5,fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)